In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy.linalg import inv
import math

In [2]:
# Two telecommunication companies, two tech companies, and Walmart
expected_return = []
tickers = ['CMCSA','T', 'AAPL', 'MSFT', 'WMT']


start_date = '2014-01-01'
end_date = '2014-12-31'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return = ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
start_date = '2015-01-01'
end_date = '2015-12-31'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
start_date = '2016-01-01'
end_date = '2016-12-30'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
    
start_date = '2017-01-01'
end_date = '2017-12-29'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)
    
start_date = '2018-01-01'
end_date = '2018-12-31'
for ticker in tickers:
    unit_cost = data.DataReader(ticker, 'yahoo', start_date)['Open'][0]
    adj_close = data.DataReader(ticker, 'yahoo', end_date)['Adj Close'][0]
    ticker_return =  ((adj_close - unit_cost) / unit_cost)*100
    expected_return.append(ticker_return)

In [3]:
expected_return = np.array(expected_return)
years = [];
for i in range(2014, 2019):
    years.append(i)

returns_df = pd.DataFrame(expected_return.reshape(5,5), columns = tickers)
returns_df['year'] = years
display(returns_df)

,CMCSA,T,AAPL,MSFT,WMT,year
0,1.705207,-28.388810,27.456902,11.344808,-3.731348,2014
1,-10.146865,-18.620589,-11.904417,9.353252,-35.554956,2015
2,18.108101,4.171407,7.569124,8.095585,6.627371,2016
3,9.813918,-20.169875,41.511218,31.603032,36.547809,2017
4,-18.278692,-32.165617,-8.887455,15.891587,-8.055473,2018


In [4]:
# Read in all datasets
df8 = pd.read_csv("2018_Financial_Data.csv")
df7 = pd.read_csv("2017_Financial_Data.csv")
df6 = pd.read_csv("2016_Financial_Data.csv")
df5 = pd.read_csv("2015_Financial_Data.csv")
df4 = pd.read_csv("2014_Financial_Data.csv")

# Insert data into list for iteration
list1 = [df8, df7, df6, df5, df4]
for i in range(5):
    
    # Modify data to get desired amount of features
    financial_data = pd.DataFrame(list1[i], columns = ['Symbol', 'Revenue', 'Revenue Growth', 'Gross Profit', 
                                                  'Operating Income', 'Earnings before Tax', 'Free Cash Flow', 
                                                  'Net Income', 'Total current assets',
                                                  'Operating Expenses',  'Net Debt', 'Short-term debt', 'Long-term debt', 
                                                  'Total shareholders equity', 'Weighted Average Shs Out', 
                                                  'Total current liabilities', 'Total debt', 'Total liabilities']) 
    nan_value = float("NaN")
    target_data = financial_data

    # Get data from each selected stock
    s1 = pd.DataFrame(financial_data.loc[financial_data.loc[:,('Symbol')] == "CMCSA", :]) #dataframes
    s2 = pd.DataFrame(financial_data.loc[financial_data['Symbol'] == "T", :])
    s3 = pd.DataFrame(financial_data.loc[financial_data['Symbol'] == "AAPL", :])
    s4 = pd.DataFrame(financial_data.loc[financial_data['Symbol'] == "MSFT", :])
    s5 = pd.DataFrame(financial_data.loc[financial_data['Symbol'] == "WMT", :])
    
    # Append stock data together throughout 5 years
    if i == 0:
        s1_total = s1
        s2_total = s2
        s3_total = s3
        s4_total = s4
        s5_total = s5
    else:
        s1_total = s1_total.append(s1)
        s2_total = s2_total.append(s2)
        s3_total = s3_total.append(s3)
        s4_total = s4_total.append(s4)
        s5_total = s5_total.append(s5)

# Add on Expected Return column to each stock

s1_total['Expected Return'] = [expected_return[0],expected_return[5], expected_return[10], 
                            expected_return[15], expected_return[20]]
s2_total['Expected Return'] = [expected_return[1],expected_return[6], expected_return[11], 
                             expected_return[16], expected_return[21]]
s3_total['Expected Return'] = [expected_return[2],expected_return[7], expected_return[12], 
                             expected_return[17], expected_return[22]]
s4_total['Expected Return'] = [expected_return[3],expected_return[8], expected_return[13], 
                             expected_return[18], expected_return[23]]
s5_total['Expected Return'] = [expected_return[4],expected_return[9], expected_return[14], 
                             expected_return[19], expected_return[24]]

In [5]:
# Create an array of the tables above
dataTables = [s1_total, s2_total, s3_total, s4_total, s5_total]

# Loop through each of the tables and calculate the pararmeters for multiple linear regrression 
# After calculating the parameters, test the regression and calcualte RMSE
for i in range(0, len(dataTables)):
    
    # Find what are the X and Y matrices for normal equation
    x = dataTables[i].drop('Symbol',1)
    x = x.drop('Expected Return', 1).to_numpy()
    y = dataTables[i]['Expected Return'].to_numpy()
    
    # Calculate the parameters
    XTX = x.transpose().dot(x)
    XTX_inverse = inv(np.matrix(XTX))
    XTX_inverse_X = XTX_inverse.dot(x.transpose())
    param_theta = XTX_inverse_X.dot(y)
    

    # Find RMSE for the model using the training data and targets we have
    error_sum = 0
    for j in range (0,5):
        y_pred = param_theta.dot(x[j])
        pred_difference = y_pred - returns_df[tickers[i]][j]
        error_sum = error_sum + ((pred_difference**2) / 5)
    RMSE = math.sqrt(error_sum)
    print("RMSE for ", tickers[i], " is ", RMSE) #print the RMSE

RMSE for  CMCSA  is  7.109597932897902
RMSE for  T  is  30.806530695333464
RMSE for  AAPL  is  29.072127998997487
RMSE for  MSFT  is  33.20945024152871
RMSE for  WMT  is  12.504245519709308


In [46]:
def get_indexes_min_value(l):
    min_value = min(l)
    if l.count(min_value) > 1:
        return [i for i, x in enumerate(l) if x == min(l)]
    else:
        return l.index(min(l))
    
# Create an array of the tables above
dataTables = [s1_total, s2_total, s3_total, s4_total, s5_total]

# Loop through each of the tables and calculate the pararmeters for multiple linear regrression 
# After calculating the parameters, test the regression and calcualte RMSE
for i in range(0, len(dataTables)):
    
    # Find what are the X and Y matrices for normal equation
    x = dataTables[i].drop('Symbol',1)
    x = x.drop('Expected Return', 1).to_numpy()
    y = dataTables[i]['Expected Return'].to_numpy()
    
    # Calculate the parameters with regularization
    XTX = x.transpose().dot(x)
    a = np.eye(len(XTX), dtype=int)
    a[0,0] = 0
    lam = np.arange(0, 100, 0.001).tolist() #range of lambda to select the best one
    error_list = []
    
    # Loop that selects the parameters and lambda that gives the lowest error
    for l in range(0, len(lam)):
        XTX_reg = XTX + lam[l]*a
        XTX_inverse = inv(np.matrix(XTX_reg))
        XTX_inverse_X = XTX_inverse.dot(x.transpose())
        param_theta = XTX_inverse_X.dot(y)
       
        error_sum = 0
        for j in range (0,5):
            y_pred = param_theta.dot(x[j])
            pred_difference = y_pred - returns_df[tickers[i]][j]
            error_sum = error_sum + ((pred_difference**2) / 5)
        RMSE = math.sqrt(error_sum)
        error_list.append(RMSE)
    lowestRMSE = min(error_list)
    optimumLambda = lam[get_indexes_min_value(error_list)]
    print("RMSE for ", tickers[i], " is ", lowestRMSE, " with parameter lambda ", optimumLambda, " for regularization.") 
    
    # Show the parameters that optimize this
    XTX_reg = XTX + optimumLambda*a
    XTX_inverse = inv(np.matrix(XTX_reg))
    XTX_inverse_X = XTX_inverse.dot(x.transpose())
    param_theta = XTX_inverse_X.dot(y)
    
    var =[]
    for k in range (0,17):
        var.append("x"+ str(k))
    parameter = pd.DataFrame(param_theta, columns = var)
    display(parameter) #show the model that is optimum for each stock

RMSE for  CMCSA  is  0.18862708051431612  with parameter lambda  28.398  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,3923.944246,10.01596,-3923.944246,1.806966e-08,-2.636624e-08,-5.659471e-08,-2.245830e-09,6.545901e-09,3.625519e-09,1.352107e-09,-6.894345e-08,-5.163137e-09,3.272581e-09,-3.996641e-08,9.520679e-10,-2.768717e-09,3.455989e-09


RMSE for  T  is  4.515556500672374  with parameter lambda  77.849  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,-5.859407e-10,47.909992,-2.548385e-09,-2.379878e-08,3.742443e-08,-2.411367e-08,-5.880645e-09,-8.413860e-09,5.893922e-09,5.745940e-09,1.924010e-08,-4.116191e-09,-1.057623e-09,1.925352e-08,-6.485037e-10,2.060359e-09,-1.038684e-10


RMSE for  AAPL  is  1.4570795363386986  with parameter lambda  79.869  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,1.164275e-09,-41.263423,-2.601908e-09,-2.611546e-09,4.589292e-09,1.945459e-09,1.320230e-09,3.312260e-09,-6.701443e-09,-2.622849e-09,4.872907e-09,-3.737355e-09,-3.920528e-10,-3.466787e-08,-5.589116e-10,2.020254e-09,-6.512611e-10


RMSE for  MSFT  is  2.3901970111435147  with parameter lambda  28.548000000000002  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,9.010823e-09,615.454315,-1.876316e-08,6.349422e-08,-6.359393e-08,1.286064e-08,-3.895942e-09,8.068148e-10,1.131193e-09,4.606465e-10,7.459663e-10,2.289706e-10,-2.756374e-10,2.383727e-09,-4.520692e-10,-5.686949e-10,-4.184172e-10


RMSE for  WMT  is  0.39274079716343163  with parameter lambda  7.615  for regularization.


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16
0,1.216464e-09,63.144323,-5.179570e-10,-6.071092e-09,5.566652e-08,-1.579148e-08,-8.900593e-08,1.351725e-08,4.067589e-09,-2.035689e-08,-6.251594e-11,9.096248e-09,-3.470943e-09,-2.183819e-07,-1.068910e-08,7.423860e-09,4.735739e-09
